# Scrape How I Built This (Podcast)

- code for pulling How I Built this transcripts from happyscribe.com
- note: there are a lot more podcasts on the official how I built this website available for download, but they are audio files, not text, and would require an anditional transcription step.

In [ ]:
# Jupyter magic
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# imports
import requests
from bs4 import BeautifulSoup
import re
import time
import random
import pandas as pd
from seleniumrequests import Chrome
import pathlib

In [ ]:
save_dir = "../1_data/how_i_built_this"
url = "https://www.happyscribe.com/public/how-i-built-this-with-guy-raz"

In [ ]:
# Change this to where the chromedriver is in your environment
path_to_chromedriver = '../chromedriver'

driver = Chrome(executable_path=path_to_chromedriver)
driver.get(url)

### Note: to get all the links, use the selinium window to scroll down repeatedly triggerint the "load more" action in the window

In [ ]:
links = driver.find_elements_by_class_name('hsp-card-episode')
len(links)

In [ ]:
episode_urls = []
for l in links:
    episode_urls.append(l.get_property('href'))
    
episode_urls[:3]

In [ ]:
def clean_text(text):
    drop_list = ["NPR sponsor", 
                 "before we start", "support of this show", "support show", 
                 "how I built this book", "Support for NPR", "our full interview",  "That's an excerpt", 
                 "question from Facebook", "Stay with us", "I'm Guy"]
    clean = []
    for t in text:
        drop = False
        for d in drop_list:
            if d in t:
                drop = True
        if drop:
            continue
        clean.append(t)
    return clean

def parse_title(title):
    
    if 'How I Built Resilience:' in title:
        t = title.split('How I Built Resilience:')[1]
        if 'of' in t:
            name =  t.split('of')[0]
            org =  t.split('of')[-1]
            return name.strip(), org.strip()
        if 'Live with':
            name = t.split('Live with')[-1]
            org = ""
    else:
        t = title.split(':')[-1]
        name = t.split('(')[0]
        org = title.split(':')[0]
    return name.strip(), org.strip()

def get_episode(episode_url):
    response = requests.get(episode_url)
    print(response)
    soup = BeautifulSoup(response.text, 'html.parser')
    title = soup.find('h1').text
    text = [t.text for t in soup.find('main').find_all('p')]
    text = clean_text(text)
    year = soup.find(id='date').text.split()[-1]
    return title, text, year


title, text, year = get_episode(episode_urls[2])


In [ ]:
episode_url = episode_urls[5]
response = requests.get(episode_url)
print(response)
soup = BeautifulSoup(response.text, 'html.parser')
episode_url

In [ ]:
parse_title(title)

In [ ]:
name, org = parse_title(title)
name, org

In [ ]:

data = []
for episode_url in episode_urls:
    title, text, year = get_episode(episode_url)

    
    name, org = parse_title(title)
    print(title)
    print(len(text), 'lines')
    
    fname = f"{name.split()[-1].lower()}_hibt_{year}"
    fpath = f"{save_dir}/{fname}.txt"
    print(fname)
    with open(fpath, 'w') as f:
        f.write("\n".join(text))
    d = {
        'name': name,
        'org': org,
        'title': title,
        'year': year,
        'filename': fname,
        'source': 'hibt',
        'url': episode_url,
    }
    data.append(d)

In [ ]:
df = pd.DataFrame(data)
df.to_csv(f"{save_dir}/index.csv")
df